In [42]:
import pandas as pd 
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np 
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity
import re


In [158]:
data=pd.read_csv(r"C:\Users\User\Desktop\my project\3  (9.11.2025)\results_with_crew.csv")

In [160]:
data.head()


,tconst,primaryTitle,startYear,rank,averageRating,numVotes,runtimeMinutes,directors,writers,genres,IMDbLink,Title_IMDb_Link
0,tt0111161,The Shawshank Redemption,1994,1,9.3,3104681,142,Frank Darabont,"Stephen King, Frank Darabont",Drama,"<a href=""https://www.imdb.com/title/tt0111161""...","<a href=""https://www.imdb.com/title/tt0111161""..."
1,tt0068646,The Godfather,1972,2,9.2,2163817,175,Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola","Crime, Drama","<a href=""https://www.imdb.com/title/tt0068646""...","<a href=""https://www.imdb.com/title/tt0068646""..."
2,tt0468569,The Dark Knight,2008,3,9.1,3079698,152,Christopher Nolan,"Jonathan Nolan, Christopher Nolan, David S. Go...","Action, Crime, Drama","<a href=""https://www.imdb.com/title/tt0468569""...","<a href=""https://www.imdb.com/title/tt0468569""..."
3,tt0167260,The Lord of the Rings: The Return of the King,2003,4,9.0,2110101,201,Peter Jackson,"J.R.R. Tolkien, Fran Walsh, Philippa Boyens, P...","Adventure, Drama, Fantasy","<a href=""https://www.imdb.com/title/tt0167260""...","<a href=""https://www.imdb.com/title/tt0167260""..."
4,tt0108052,Schindler's List,1993,5,9.0,1547957,195,Steven Spielberg,"Thomas Keneally, Steven Zaillian","Biography, Drama, History","<a href=""https://www.imdb.com/title/tt0108052""...","<a href=""https://www.imdb.com/title/tt0108052""..."


In [78]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   movie_name      5000 non-null   object        
 1   year            5000 non-null   datetime64[ns]
 2   rank            5000 non-null   int64         
 3   averageRating   5000 non-null   float64       
 4   runtimeMinutes  5000 non-null   int64         
 5   directors       5000 non-null   object        
 6   writers         4975 non-null   object        
 7   genres          5000 non-null   object        
 8   IMDbLink        5000 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 351.7+ KB


In [108]:
data.drop(["tconst","numVotes","Title_IMDb_Link"], axis=1, inplace=True)

data= data.rename(columns={
    'primaryTitle':'movie_name',
    'startYear' : 'year',
})

data["writers"] = data["writers"].fillna("unknown_writer")

KeyError: "['tconst', 'numVotes', 'Title_IMDb_Link'] not found in axis"

In [123]:
data["writers"] = data["writers"].fillna("unknown_writer")


In [150]:
data['year'] = data['year'].astype(str).str.extract(r'(\d{4})')[0]


In [156]:
data["genres"]=data["genres"].str.replace(",", " ")
data["writers"]=data["writers"].str.replace(",", " ")
data["directors"]=data["directors"].str.replace(",", " ")
data["combined_feature"] = (
    data["genres"] + " " + 
    data["directors"] + " " + 
    data["writers"] + " " +
    data["year"].astype(str) + " " +
    "rating_" + data["averageRating"].round(1).astype(str) + " " +
    "runtime_" + data["runtimeMinutes"].astype(str)
)

data.head(3)

,movie_name,year,rank,averageRating,runtimeMinutes,directors,writers,genres,IMDbLink,combined_feature,weighted_combined_feature
0,The Shawshank Redemption,1994,1,9.3,142,Frank Darabont,Stephen King Frank Darabont,Drama,"<a href=""https://www.imdb.com/title/tt0111161""...",Drama Frank Darabont Stephen King Frank Darab...,genre_Drama genre_Drama genre_Drama director_F...
1,The Godfather,1972,2,9.2,175,Francis Ford Coppola,Mario Puzo Francis Ford Coppola,Crime Drama,"<a href=""https://www.imdb.com/title/tt0068646""...",Crime Drama Francis Ford Coppola Mario Puzo ...,genre_Crime genre_Crime genre_Crime genre_Dram...
2,The Dark Knight,2008,3,9.1,152,Christopher Nolan,Jonathan Nolan Christopher Nolan David S. Go...,Action Crime Drama,"<a href=""https://www.imdb.com/title/tt0468569""...",Action Crime Drama Christopher Nolan Jonatha...,genre_Action genre_Action genre_Action genre_C...


In [154]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   movie_name                 5000 non-null   object 
 1   year                       5000 non-null   object 
 2   rank                       5000 non-null   int64  
 3   averageRating              5000 non-null   float64
 4   runtimeMinutes             5000 non-null   int64  
 5   directors                  5000 non-null   object 
 6   writers                    5000 non-null   object 
 7   genres                     5000 non-null   object 
 8   IMDbLink                   5000 non-null   object 
 9   combined_feature           5000 non-null   object 
 10  weighted_combined_feature  5000 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 429.8+ KB


In [131]:
data.isnull().sum()

movie_name          0
year                0
rank                0
averageRating       0
runtimeMinutes      0
directors           0
writers             0
genres              0
IMDbLink            0
combined_feature    0
dtype: int64

In [133]:
vectorize = TfidfVectorizer(stop_words="english", max_features=50000 )
vector = vectorize.fit_transform(data["combined_feature"])

vector

<5000x8444 sparse matrix of type '<class 'numpy.float64'>'
	with 59683 stored elements in Compressed Sparse Row format>

In [136]:
similarities = cosine_similarity(vector, dense_output=False)
print(similarities.shape)


(5000, 5000)


In [138]:
def create_weighted_feature(row):
    features = []
    
    # ژانر - وزن بالا
    genres = row['genres'].replace(',', ' ')
    for genre in genres.split():
        features.extend([f"genre_{genre}"] * 3)  # تکرار ۳ بار
    
    # کارگردان - وزن متوسط
    directors = row['directors'].replace(',', ' ')
    for director in directors.split()[:2]:  # فقط ۲ کارگردان اول
        features.extend([f"director_{director}"] * 2)
    
    # نویسنده - وزن کم
    writers = row['writers'].replace(',', ' ')
    for writer in writers.split()[:2]:  # فقط ۲ نویسنده اول
        features.append(f"writer_{writer}")
    
    # سایر ویژگی‌ها
    features.extend([
        f"year_{row['year'].year}",
        f"rating_{row['averageRating']:.1f}",
        f"runtime_{row['runtimeMinutes']}"
    ])
    
    return ' '.join(features)

data["weighted_combined_feature"] = data.apply(create_weighted_feature, axis=1)

In [144]:
def recommend_movie(movie_title, k=5, min_rating=7.0):
    try:
        # پیدا کردن فیلم
        movie_idx = data[data["movie_name"].str.strip().str.lower() == movie_title.strip().lower()].index[0]
        
        # محاسبه شباهت‌ها
        similarities_score = list(enumerate(similarities[movie_idx]))
        sorted_scores = sorted(similarities_score, key=lambda x: x[1], reverse=True)
        
        # فیلتر و انتخاب
        recommendations = []
        for idx, score in sorted_scores[1:k*2]:  # بیشتر بگیریم تا بعد فیلتر کافی داشته باشیم
            if len(recommendations) >= k:
                break
            if data.iloc[idx]["averageRating"] >= min_rating and idx != movie_idx:
                recommendations.append({
                    'title': data.iloc[idx]["movie_name"],
                    'rating': data.iloc[idx]["averageRating"],
                    'similarity_score': round(score, 3)
                })
        
        if not recommendations:
            return "هیچ پیشنهادی پیدا نشد—فیلتر رو چک کن."
        
        return recommendations
        
    except IndexError:
        return "فیلم پیدا نشد—عنوان دقیق وارد کن."

In [ ]:
import streamlit as st

# UI ساده
st.title("سیستم پیشنهاد فیلم")
st.write("جستجوی فیلم و دریافت پیشنهاد با داده از Kaggle")

# ورودی‌ها
movie_title = st.text_input("عنوان فیلم را وارد کنید (مثال: The Godfather):")
k = st.slider("تعداد پیشنهادها:", min_value=1, max_value=5, value=3)  # محدوده ساده
min_rating = st.slider("حداقل ریتینگ:", min_value=5.0, max_value=9.0, value=7.0, step=0.5)  # گام بزرگ‌تر

# دکمه و نمایش نتیجه
if st.button("جستجو"):
    if movie_title:
        recommendations = recommend_movie(movie_title, k, min_rating)
        if isinstance(recommendations, list):
            st.table(pd.DataFrame(recommendations))  # نمایش جدول ساده
        else:
            st.error(recommendations)
    else:
        st.warning("لطفاً عنوان فیلم را وارد کنید!")